In [1]:
%config InlineBackend.figure_format = 'retina'

from contextlib import contextmanager

import pandas
from IPython.display import display, HTML

display(HTML("<style>.container{ width:100% !important; } div.prompt{ min-width: 10ex; } </style>"))
display(HTML("""<table>
<tr><th>Each cell:
<tr><td>
Size in kb<br>
Megapixels/s<br>
ssimulacra2 mectric
</table>
"""))

def read_df(filename):
    df = pandas.read_csv(f'./datasets/{filename}')
    df.sort_values(['fname', 'effort', 'codec'], inplace=True)
    df['size_kb'] = df['size'] / 1000
    df['data'] = df.agg('<tt>{0[size_kb]:5.2f}<br>{0[perf]:5.2f}<br>{0[dssim]:.0f}</tt>'.format, axis=1)
    return df


def chunks(lst, n):
    n = n or len(lst)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


@contextmanager
def floats(up_to=None):
    tables = []
    yield tables.append
    for chunk in chunks(tables, up_to):
        display(HTML("".join(
            f'<div style="float:left">{table}</div>'
            for table in chunk
        )))


def show_run(df, up_to=None):
    with floats(up_to=up_to) as append:
        for fname in df.fname.unique():
            table = df[df.fname == fname]
            table = table.pivot(index='quality', columns=['codec', 'effort'], values='data')
            append(f"<h2>{fname}</h2>" + table.to_html(escape=False))


def compare(data):
    frames = []
    for filename, suffix in data:
        df = read_df(filename)
        df.fname = df.fname + suffix
        frames.append(df)

    df = pandas.concat(frames, ignore_index=True)
    df.sort_values(['fname', 'effort', 'codec'], inplace=True)
    return df


def h1(title):
    display(HTML(f"<h1>{title}</h1>"))

In [2]:
h1("Compare AOM speeds")

show_run(read_df('libavif.all_speeds.csv'))

In [3]:
h1("Compare subsampling")

show_run(read_df('libavif.subsampling.csv'))

codec,ss:0,ss:1,ss:2,ss:0,ss:1,ss:2
effort,5,5,5,6,6,6
quality,,,,,,
10,33.02 0.8489,30.83 1.3187,27.67 1.1486,36.52 3.9588,33.45 4.4786,30.55 4.8285
12,29.40 0.8988,27.67 1.3387,24.94 1.1685,32.68 4.1187,30.19 4.4685,27.42 4.6384
14,26.92 0.9288,25.32 1.4286,22.86 1.2084,29.66 4.2886,27.47 4.5785,25.24 4.8783
16,24.78 0.9687,23.62 1.4385,21.21 1.1984,27.22 4.3685,25.39 4.7684,23.28 4.8882
18,23.13 0.9686,22.02 1.4685,19.85 1.2283,25.47 4.4385,23.97 4.6283,21.94 5.0282
20,21.53 0.9585,20.64 1.4184,18.44 1.2482,23.85 4.5284,22.33 4.8382,20.60 5.0081
22,20.32 0.9585,19.49 1.3583,17.59 1.1882,21.80 3.3883,20.85 4.1982,19.13 3.9780
24,19.19 0.9184,18.46 1.3483,16.40 1.1881,20.59 3.2283,19.63 4.0181,17.99 3.6980


In [4]:
h1("Compare with webp")

show_run(read_df('libavif.compare_webp.csv'))

codec,libavif,webp,libavif,webp,libavif,webp
effort,5,5,6,6,7,7
quality,,,,,,
8,38.48 0.8190,107.3810.6786,42.81 3.9288,107.3810.6786,48.61 4.6289,107.3810.6786
12,29.40 0.8388,107.3810.6786,32.68 3.4387,107.3810.6786,34.08 4.1987,107.3810.6786
16,24.78 0.9187,107.3810.6786,27.22 4.2685,82.9211.0085,27.85 5.9385,78.1610.9385
20,21.53 0.9885,85.7611.4985,23.85 4.5784,51.9911.5084,23.75 6.1883,45.86 9.9583
24,19.19 0.9484,62.6311.5784,20.59 3.4883,45.86 9.9583,20.76 6.2781,38.0011.8481
28,16.23 0.9682,41.7410.6682,17.52 3.4480,34.2512.2380,17.73 5.3876,28.9712.5676
32,13.83 0.9979,33.1512.3879,14.90 3.4077,29.6912.3777,15.17 6.4569,21.4812.3969
36,11.61 1.0276,28.9712.5676,12.55 3.7473,24.8312.5773,12.60 6.7564,17.5312.3864


In [5]:
h1("Compare CPUs")

df = compare([
    ('libavif.m1.csv', ' m1'),
    ('libavif.m1.x86.csv', ' m1.x86'),
    ('libavif.c6a.csv', ' c6a')
])
show_run(df, up_to=3)